In [16]:
import gzip
import matplotlib.pyplot as plt
#matplotlib inline
import numpy as np
import numpy.ma as ma
from numpy import matmul as mul
import scipy as sp
from sklearn import manifold
from sklearn import neighbors
from sklearn import datasets
import struct

In [17]:
# Parse data from zipped mnist files into numpy arrays
# Modified from source: https://gist.github.com/tylerneylon/ce60e8a06e7506ac45788443f7269e40
#
# Input:  filename --> Zipped file to parse
#
# Output: return   --> Numpy array of unint8 data points
#
def read_idx(filename):
    with gzip.open(filename) as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

In [18]:
# Construct an nxm matrix of evenly distributed samples from an input sample set
#
# Inputs: Y    --> Sample matrix
#         n, m --> Landmark matrix dimensions
#
# Output: idx  --> nxm Landmark matrix
#
def find_landmarks(Y, n, m):
    xr = np.linspace(np.min(Y[:,0]), np.max(Y[:,0]), n)
    yr = np.linspace(np.min(Y[:,1]), np.max(Y[:,1]), m)
    xg, yg = np.meshgrid(xr, yr)
    idx = [0]*(n*m)
    for i, x, y in zip(range(n*m), xg.flatten(), yg.flatten()):
        idx[i] = int(np.sum(np.abs(Y-np.array([x,y]))**2, axis=-1).argmin())
    return idx

In [19]:
# Compute the complete nearest neighbors array
#
# Inputs: X      --> training data
#         n_nbrs --> the number of nearest neighbors to search for
#         n_comp --> the number of components
def compute_nearest_neighbors(X, n_nbrs, n_comp):
    print("Computing nearest neighbors...")
    tree = neighbors.BallTree(X, leaf_size=n_comp)
    dist, ind = tree.query(X, k=n_nbrs)
    print("Complete nearest neighbor index: ", np.shape(ind))
    return dist, ind

In [20]:
#------------------------------------------------------------------------------
# Inputs: x --> the basis vector (784 x 1)
#         E --> eta is the matrix of nearest neighbors (k x 784)
# 
# Outputs: C --> the covariance matrix of the nearest neighbors of the vector x 
#
# Iterate theough the nearest neighbors and compute the  
#
def calc_covariance(Eta):
    #print("Calculate covariance:")
    #print("Eta: ", np.shape(Eta))
    
    # Compute the local covariance matrix
    #C = np.cov(Eta)
    C = mul(Eta.T, Eta)
    #print("Local covariance matrix C: ", np.shape(C))

    # Regularize the covariance matrix because otherwise it is singular
    I = np.identity(np.shape(C)[0])
    eps = .001 * np.trace(C)
    C = C + eps * I
    
    return C

In [21]:
#------------------------------------------------------------------------------
# Scale weight values for nearest neighbors
#
# Inputs:  wght --> the wight vector with non-neighbors blanked out
# 
# Outputs: w    --> the adjusted weight vector    
#
def scale_neighbors(wght):
    sumw = np.sum(wght)
    w = wght / sumw
    #print("Sum of weights: ", np.sum(w))
    return w

In [22]:
#------------------------------------------------------------------------------
# Build the matrix of nearest neighbors
# 
# Inputs: X    --> the source data matrix X to get the neighbors from
#         nbrs --> the list of neighbor indices
#
# Outputs: Eta --> the nearest neighbor matrix for Xi
#
# Use the neighbors list to get the corresponding vector from X and put the 
# vectors together in a nearest neighbor matrix
#
def build_nbrs_matrix(X, nbrs, k):
    #print("Build nearest neighbors matrix...")
    # Build a matrix of the nearest neighbors
    nbrs = np.delete(nbrs, 0, axis=0)   # remove Xi
    print("nbrs: ", np.shape(nbrs))
    
    D = np.shape(X)[1]                # set number of rows for Eta
    N = np.shape(nbrs)[1]         # set columns for Eta. Subtract 1 because index from ball tree includes the vector itself
    Eta = np.zeros([D, k])
    
    #print("D: ", D, " N: ", N)
    #print("Eta: ", np.shape(Eta))
    
    # Get each row vector from X corresponding to the nearest neighbor
    for n in range(1, N):
        Eta[:,n-1] = X[nbrs[0][n]]
        #print("nbr ", nbrs[0][n])
        
        
    #print("Eta: ", np.shape(Eta))    
    #print(Eta)
    return Eta


In [23]:
#------------------------------------------------------------------------------
# Center the matrix
#
#
#
#
def center_nbrs_matrix(x, Eta):
    #print("Center the neighbors matrix...")
    n = np.shape(Eta)[1]
    d = np.shape(x)[0]
    #print("n: ", n)
    #print("d: ", d)
    #print("x: ", np.shape(x)) 
    sub = np.tile(x, (n,1))
    #print("sub: ", np.shape(sub))
    #print("Eta: ", np.shape(Eta))
    eta = Eta - sub.T
    return eta

In [10]:
#------------------------------------------------------------------------------
# Determine the reconstruction weights for each X
#
# Inputs:  X    --> Data Matrix
#          i    --> Index for vector xi we want to construct weights for
#          tree --> nearest neighbors tree
#          k    --> number of nearest neighbors  
# 
# Outputs: W    --> the weight matrix
#
def construct_weight_vector(X, i, nbrs, k):
    # Setup matrices and variables
    #print("Construct weight matrix")
    #print("k = ", k)
    #print("Image #", i)
    #print("Neighbors = ", nbrs)

    # Build the matrix of nearest neighbors
    Eta = build_nbrs_matrix(X, nbrs, k)
    
    # Center the data
    Eta = center_nbrs_matrix(X[i,:], Eta)
    
    # Compute local covariance matrix for the vector v in X and all neighbors
    C = calc_covariance(Eta)

    # Solve the linear system with constraint that rows of weights sum to one
    b = np.ones(np.shape(C)[1])  # build the constant vector
    w = np.linalg.solve(C, b)    # compute solution
    
    # Apply first constraint to zero out non-neighbor weights
    #w = clear_non_neighbors(nbrs, w)
    
    # Apply second constraint to scale valid neighbors
    w = scale_neighbors(w)
    
    #print("Weight vector constructed...  w: ", np.shape(w))
    print("w = ", w)
    
    return w

In [11]:
#------------------------------------------------------------------------------
# Calculate the sum of elements in a weight row for matrix M
#
# Inputs:   W
#             
#
#
#
#
def sum_weight_row(W, i, j):
    s = 0
    for k in range(0, np.shape(W)[0]):
        s = s + W[k,i] * W[k,j] 
    return s

In [12]:
#------------------------------------------------------------------------------
#
#
#
#
#
def compute_embedding_components(W, d):
    #print("Compute embedding components:")
    #create sparse matrix M = (I-W)'*(I-W)
    
    # Create matrix M
    print("Create matrix M...")
    M = np.zeros(np.shape(W))
    d = np.shape(M)[0]
    n = np.shape(M)[1]
    
    print("d: ", d, " n: ", n)
    
    for i in range(0, d):
        for j in range(0, n):
            if i == j:
                delta = 1
            else:
                delta = 0
            M[i,j] = delta - W[i,j] - W[i,j] + sum_weight_row(W, i, j)
        print("M[",i,",",j,"] = ", M[i,j])        
    

    # 
    U, S, Vt = np.linalg.svd(M)
    
    #print("U: ", np.shape(U))
    #print(U,"\n")
    #print("S: ", np.shape(S))
    #print("Vt: ", np.shape(Vt))
    
    d = np.shape(U)[1]
    E = U[:,d-3:d-1]
    print("E: ", np.shape(E))
    print(E)
    
    Y = np
    
    return Y 

In [13]:
#------------------------------------------------------------------------------
# TODO - We only have to implement this function, swap it with the manifold one below
# Seek a low-rank projection on an input matrix
#
# Inputs: X            --> Input matrix to reduce
#         n_neighbors  --> Maximum number of neighbors used for reconstruction
#         n_components --> Maximum number of linearly independent components for reconstruction
#
# Output: Y            --> Reconstructed vectors in a lower rank
#         err          --> (Optional implementation) Error margin of vectors
#
def locally_linear_embedding(X, n_neighbors, n_components):
    
    # Step 1: Find the nearest neighbors at for each sample
    
    # Testing
    #tree = neighbors.BallTree(X, leaf_size=n_components)
    #dist, ind = tree.query(X[:1], k=n_neighbors)
    #print("ind: ", ind)
    
    # Primetime
    #n_neighbors = n_neighbors + 1
    dist, ind = compute_nearest_neighbors(X, n_neighbors + 1, n_components)
    
    # Step 2: Construct the weight matrix
    D = np.shape(X)[0]
    N = np.shape(X)[1]
    W = np.zeros((n_neighbors, D))  # subtract 1 from D because nearest neighbor returns the vector itself
    
    print("X: ", np.shape(X))
    print("N: ", N, " D: ", D)
    print("W: ", np.shape(W))

    # Loop through each image and construct its weight matrix 
    for i in range(0, D):
        print("Image: ",i)
        w = construct_weight_vector(X, i, ind, n_neighbors)
        W[:,i] = w[:]
    
    # Step 3: Compute vectors that are reconstructed by weights
    Y = compute_embedding_components(W, 2)
    
    # Calculate the error
    
    err = 0.001    # placeholder
    return Y, err

In [14]:
# MAIN
# -----------------------------------------------------------------------------
np.set_printoptions(precision=2, edgeitems=10)

# Extract mnist data from files
raw_train = read_idx("train-images-idx3-ubyte.gz")
train_data = np.reshape(raw_train, (60000, 28*28))
train_label = read_idx("train-labels-idx1-ubyte.gz")

In [15]:
# Train algorithm and calculate landmark graph
X = train_data[train_label == 8]

# Test by using scikit class
#Y, err = manifold.locally_linear_embedding(X, n_neighbors=10, n_components=2)

# Verify by using our custom method
Y, err = locally_linear_embedding(X, n_neighbors=10, n_components=2)


landmarks = find_landmarks(Y, 5, 5)

Computing nearest neighbors...
Complete nearest neighbor index:  (5851, 11)
X:  (5851, 784)
N:  784  D:  5851
W:  (10, 5851)
Image:  0
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  0.9999999999999999
w =  [-0.07  0.08  0.14  0.16 -0.16  0.33 -0.03  0.59  0.03 -0.07]
Image:  1
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0000000000000002
w =  [ 0.28  0.18  0.22  0.05  0.22  0.15  0.03  0.08 -0.05 -0.16]
Image:  2
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  0.9999999999999998
w =  [ 0.13  0.06 -0.07  0.37 -0.34  0.29  0.05  0.5  -0.01  0.03]
Image:  3
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of wei

w =  [ 0.35  0.11  0.05  0.29 -0.14  0.33  0.02  0.01 -0.14  0.12]
Image:  45
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.48 -0.33  0.09  0.23 -0.07  0.65 -0.02  0.06  0.06 -0.15]
Image:  46
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.15  0.21  0.21  0.35 -0.24 -0.03  0.05  0.12 -0.05  0.21]
Image:  47
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [-0.21  0.61  0.12 -0.04 -0.21  0.48  0.21 -0.01  0.15 -0.1 ]
Image:  48
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.17 -0.11 -0.16  0.22 -0.47  0.36  0.3   0.29  0.33  0.07]
Image:  49
Construct 

Center the neighbors matrix...
Calculate covariance:
Sum of weights:  0.9999999999999999
w =  [ 0.03  0.5  -0.36  0.22 -0.12  0.27  0.22  0.19 -0.08  0.13]
Image:  139
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [-6.30e-02  3.45e-01  4.42e-01  1.46e-01  3.12e-03  2.45e-04  3.01e-01
 -1.69e-01  3.49e-01 -3.54e-01]
Image:  140
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.03  0.38 -0.34  0.27 -0.35 -0.06  0.51  0.45  0.02  0.09]
Image:  141
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  0.9999999999999998
w =  [ 0.11  0.31 -0.34  0.34 -0.36 -0.13  0.45  0.09  0.36  0.17]
Image:  142
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the ne

Image:  194
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.09 -0.04 -0.3   0.13 -0.15  0.12  0.03  0.45  0.29  0.39]
Image:  195
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 1.49e-01  3.20e-01 -2.51e-01  2.15e-04 -4.21e-01  2.33e-01  4.48e-01
  5.03e-03  2.78e-01  2.38e-01]
Image:  196
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0000000000000002
w =  [ 0.2   0.02 -0.2   0.32 -0.55  0.15  0.36  0.44  0.23  0.03]
Image:  197
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  0.9999999999999999
w =  [ 0.17  0.1  -0.3   0.11 -0.18  0.31  0.37  0.25  0.21 -0.04]
Image:  198
C

w =  [-0.06 -0.09 -0.46  0.06  0.31  0.37  0.07  0.18  0.64 -0.01]
Image:  243
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.23  0.22 -0.14  0.21 -0.49  0.31  0.24 -0.01  0.17  0.27]
Image:  244
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.54 -0.14 -0.13 -0.09  0.34  0.37  0.01  0.01  0.1  -0.02]
Image:  245
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.24 -0.04 -0.12  0.1  -0.27  0.3   0.19  0.24  0.08  0.29]
Image:  246
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.21 -0.22 -0.35  0.69 -0.06  0.12  0.28 -0.13  0.13  0.32]
Image:  247
Const

...

Image:  5723
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.21 -0.19 -0.52  0.31 -0.41  0.47  0.3   0.21  0.2   0.43]
Image:  5724
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.02  0.11 -0.49  0.47 -0.29  0.05 -0.04  0.28  0.36  0.52]
Image:  5725
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.22 -0.02 -0.38  0.28 -0.33  0.51 -0.    0.28  0.03  0.42]
Image:  5726
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.08 -0.17 -0.47  0.36 -0.28  0.21  0.14  0.3   0.26  0.59]
Image:  5727
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5

w =  [ 0.    0.24  0.33  0.31 -0.2   0.28 -0.04  0.07  0.13 -0.13]
Image:  5773
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.38  0.31  0.54 -0.18 -0.11  0.18 -0.07  0.13  0.17 -0.36]
Image:  5774
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  0.9999999999999999
w =  [ 0.04  0.04 -0.    0.11  0.01  0.61 -0.15  0.44  0.09 -0.19]
Image:  5775
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.07  0.32 -0.36  0.38 -0.25  0.11  0.09  0.17  0.24  0.23]
Image:  5776
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [ 0.02  0.37 -0.46  0.22 -0.14  0.28  0.39  0.04  0.11  0.17

Center the neighbors matrix...
Calculate covariance:
Sum of weights:  0.9999999999999999
w =  [ 0.18  0.13 -0.27  0.32 -0.31  0.28 -0.04  0.33  0.22  0.16]
Image:  5808
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [-0.04  0.31 -0.57  0.44 -0.04 -0.13  0.02  0.3   0.34  0.36]
Image:  5809
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  0.9999999999999998
w =  [-0.    0.15 -0.41  0.39 -0.32  0.31  0.18  0.53  0.2  -0.03]
Image:  5810
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0000000000000002
w =  [ 0.18  0.22 -0.25  0.03 -0.21  0.27  0.03  0.48  0.12  0.13]
Image:  5811
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calc

Sum of weights:  1.0
w =  [-0.09  0.15  0.15  0.4  -0.38 -0.18  0.31  0.06  0.18  0.39]
Image:  5849
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  1.0
w =  [-0.01  0.19 -0.18  0.02 -0.28  0.67  0.2   0.19  0.16  0.04]
Image:  5850
Construct weight matrix
Build nearest neighbors matrix...
nbrs:  (5850, 11)
Center the neighbors matrix...
Calculate covariance:
Sum of weights:  0.9999999999999998
w =  [-0.09  0.55  0.11 -0.06 -0.17 -0.12  0.29  0.18  0.48 -0.17]
Compute embedding components:
Create matrix M...
d:  10  n:  5851
M[ 0 , 5850 ] =  0.33907569556232964
M[ 1 , 5850 ] =  -1.0352826014499226
M[ 2 , 5850 ] =  -0.11038959739588586
M[ 3 , 5850 ] =  0.3373714088132232
M[ 4 , 5850 ] =  0.3956812897613276
M[ 5 , 5850 ] =  0.3930778735836231
M[ 6 , 5850 ] =  -0.6209197034159102
M[ 7 , 5850 ] =  -0.2806879841848609
M[ 8 , 5850 ] =  -0.6957968768909667
M[ 9 , 5850 ] =  0.4170348318292639
E:  

NameError: name 'Y' is not defined

In [ ]:
# Plot the clustered data with landmarks overlaid
plt.scatter(Y[:,0], Y[:,1])
plt.scatter(Y[landmarks,0], Y[landmarks,1])

# Show the landmark samples in a 5x5 grid
fig = plt.figure(figsize=(15,15))
for i in range(len(landmarks)):
    ax = fig.add_subplot(5, 5, i+1)
    imgplot = ax.imshow(np.reshape(X[landmarks[i]], (28,28)), cmap=plt.cm.get_cmap("Greys"))
    imgplot.set_interpolation("nearest")
plt.show()